## This section describes the computational pipeline developed to quantify spatial partitioning of fluorescent markers in giant unilamellar vesicles (GUVs) exhibiting liquid-disordered (Ld) and liquid-ordered (Lo) phase separation. The method leverages dual-channel confocal microscopy data, where the first channel highlights vesicle membranes, and the second channel reports marker distribution (e.g., DNA). The pipeline was implemented in Python using scikit-image, OpenCV, and pandas.

### 1. Vesicle detection and preproceseeing: Gaussian Smoothing, Canny Edge Detetcion, Hough Circle Transform
### 2. Mask Generation and Refinement: Compute mask1 (ring mask) with enlargement and subtracting inner radius; refined by mask 4 (Marker refinement with DNA channel based on a fitted circle) to create the final ring mask, ensuring analysis is restricted to marker-enriched zones.
### 3. Ld/Lo Segmentation: Using Angular Gap Analysis with a filter to define the dominant Ld arc by identifying the largest angular discontinuity. 
### 4. Postprocessing and Measuremnets: Morphological Smoothing: The Ld/Lo masks are smoothed using closing  (disk radius = 3) to remove artifacts, followed by skeletonization and dilated (disk radius = 2) to smaple intensities along membrane contours. The Quantitative Analysis focueses on the Mean intensity in Ld/Lo regions. 
### 5. Visualization: Overlay plots and labels


In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.filters import gaussian, threshold_otsu
from skimage.morphology import binary_dilation, binary_erosion, disk, closing, label, remove_small_objects, skeletonize
from skimage.measure import regionprops
from scipy.ndimage import label
from scipy.optimize import least_squares

# Define Paths and Output Settings
# -------------------------------
# Set the input folder containing multi-channel TIFF images and create an output folder for processed data
image_folder = "/Users/herbert/Library/CloudStorage/OneDrive-epfl.ch/PBL/1.3_Lipids_partitioning/Paper2_PS/0Confocal/F3/3.2_Ion_mix"
newpath = image_folder + "_processed_1/"
file_output = newpath + "Analysis.csv"

# Create the output directory if it doesn’t exist
if not os.path.exists(newpath):
    os.makedirs(newpath)

# Counter to track the number of processed images
image_counter = 0

# Adjustable Parameters
# ---------------------
# Define parameters that can be tuned for vesicle detection and segmentation
enlargement_factor = 1.2  # Factor to enlarge detected radius for ring mask (20% larger)
inner_percentage = 0.7    # Percentage of enlarged radius to define inner boundary of ring mask
threshold_method = 'otsu' # Method for thresholding Ld/Lo regions; options: 'otsu', 'mean', 'median', 'percentile', 'fixed', 'mean_plus_std'
fixed_threshold = 100     # Fixed threshold value if 'fixed' method is chosen

# Threshold Computation Function
# ------------------------------
# Function to compute threshold based on the selected method for segmenting Ld/Lo phases
def compute_threshold(pixels, method, fixed_threshold=None):
    """Compute threshold based on selected method."""
    if len(pixels) == 0:
        return 0  # Default threshold when no pixels are available
    if method == 'otsu':
        return threshold_otsu(pixels)
    elif method == 'mean':
        return np.mean(pixels)
    elif method == 'median':
        return np.median(pixels)
    elif method == 'percentile':
        return np.percentile(pixels, 75)
    elif method == 'fixed':
        if fixed_threshold is None:
            raise ValueError("Fixed threshold value must be provided")
        return fixed_threshold
    elif method == 'mean_plus_std':
        return np.mean(pixels) + np.std(pixels)  
    else:
        raise ValueError(f"Unknown threshold method: {method}")

# Circle Fitting Function
# -----------------------
# Fit a circle to a set of points using least squares optimization
def fit_circle(points):
    """Fit a circle to the given points using least squares."""
    def calc_R(xc, yc):
        return np.sqrt((points[:, 0] - xc)**2 + (points[:, 1] - yc)**2)

    def f_2(c):
        Ri = calc_R(*c)
        return Ri - Ri.mean()

    center_estimate = np.mean(points, axis=0)
    result = least_squares(f_2, center_estimate)
    center = result.x
    xc, yc = center
    Ri = calc_R(xc, yc)
    R = Ri.mean()
    return xc, yc, R

# Main Processing Loop
# --------------------
# Iterate through subfolders and process TIFF images containing "-d84" in the name
for library in os.listdir(image_folder):
    if "-d84" in library:
        nr = 0
        for image_filename in os.listdir(image_folder + "/" + library):
            if "ome.tif" in image_filename:
                nr += 1
                image_counter += 1
                # Read the multi-channel TIFF image
                image_multi = imread(image_folder + "/" + library + "/" + image_filename)

                # Extract channels: Channel 0 for vesicle membranes, Channel 1 for DNA markers
                image = image_multi[:][:][0].copy()  # First channel (Liss Rhod - Liquid disorder channel)
                DNA_image = image_multi[:][:][1].copy()  # Second channel (Cy5 - DNA channel)
                print(f"Processing {library}/{image_filename}")

                # Vesicle Detection and Preprocessing
                # -----------------------------------
                # Apply Gaussian smoothing to reduce noise and Canny edge detection to find vesicle edges
                blurred_image = gaussian(image, sigma=1.5, preserve_range=True)
                edges = cv2.Canny(np.uint8(blurred_image), 40, 120)

                # Detect circular vesicles using Hough Circle Transform
                circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=1, minDist=80,
                                           param1=50, param2=30, minRadius=10, maxRadius=100)
                if circles is None:
                    print(f"No circles detected in {image_filename}")
                    continue
                circles = circles[0]

                # Mask Generation for DNA Channel
                # -------------------------------
                # Process DNA channel to create a mask for refining the ring mask
                blurred_DNA_image = gaussian(DNA_image, sigma=2, preserve_range=True)
                thresh_DNA = threshold_otsu(blurred_DNA_image)
                mask4 = blurred_DNA_image > thresh_DNA
                mask4 = binary_dilation(mask4, disk(0))

                # Initialize Data Structures
                # --------------------------
                # Set up arrays and DataFrame for storing masks and measurements
                labeled_mask = np.zeros_like(image, dtype=np.uint16)
                df_final = pd.DataFrame()
                all_final_ring_mask = np.zeros_like(image, dtype=bool)
                all_mask2 = np.zeros_like(image, dtype=bool)  # Ld regions
                all_mask3 = np.zeros_like(image, dtype=bool)  # Lo regions

                # Process Each Detected Vesicle
                # -----------------------------
                for i, (x, y, r) in enumerate(circles, start=1):
                    # Define Ring Mask
                    # ----------------
                    # Create a ring-shaped mask around the vesicle with an enlarged outer radius and smaller inner radius
                    enlarged_radius = r * enlargement_factor
                    inner_radius = enlarged_radius * inner_percentage
                    yy, xx = np.ogrid[:image.shape[0], :image.shape[1]]
                    outer_mask = (yy - y)**2 + (xx - x)**2 <= enlarged_radius**2
                    inner_mask = (yy - y)**2 + (xx - x)**2 <= inner_radius**2
                    ring_mask = outer_mask & ~inner_mask

                    # Refine Ring Mask Using DNA Channel
                    # -----------------------------------
                    # Use DNA channel mask (mask4) to refine the ring mask to marker-enriched zones
                    valid_pixels_mask = mask4 & ring_mask
                    valid_points = np.column_stack(np.where(valid_pixels_mask))[:,[1,0]]

                    if len(valid_points) > 3:  # Need at least 3 points to fit a circle
                        # Fit a circle to DNA-enriched points for a more accurate mask
                        xc_fit, yc_fit, r_fit = fit_circle(valid_points)
                        final_ring_mask = ((yy - yc_fit)**2 + (xx - xc_fit)**2 <= r_fit**2) & \
                                          ((yy - yc_fit)**2 + (xx - xc_fit)**2 > (r_fit - 1)**2)
                    else:
                        # Use original ring mask if insufficient points for fitting
                        final_ring_mask = ring_mask

                    if np.any(final_ring_mask):
                        labeled_mask[final_ring_mask] = i

                    # Ld/Lo Segmentation
                    # ------------------
                    # Segment Ld (liquid-disordered) and Lo (liquid-ordered) phases within the ring mask
                    pixels_in_ring = image[ring_mask]
                    if len(pixels_in_ring) > 0:
                        threshold = compute_threshold(pixels_in_ring, threshold_method, fixed_threshold)
                        bright_pixels_mask = pixels_in_ring > threshold

                        # Create a full-image mask for bright pixels (Ld regions)
                        bright_mask_full = np.zeros_like(image, dtype=bool)
                        ring_yy, ring_xx = np.where(ring_mask)
                        bright_mask_full[ring_yy, ring_xx] = bright_pixels_mask
                        
                        # Label connected regions and filter out small objects
                        structure = np.ones((3, 3), dtype=int)  # 8-connected structure
                        labeled_bright, num_labels = label(bright_mask_full, structure=structure)
                        
                        if num_labels > 0:
                            min_arc_size = 50  # Minimum size to filter small clusters
                            filtered_bright = remove_small_objects(labeled_bright, min_size=min_arc_size)
                            valid_pixels = filtered_bright[ring_yy, ring_xx] > 0
                        else:
                            valid_pixels = np.zeros_like(bright_pixels_mask, dtype=bool)
                    else: 
                        valid_pixels = np.zeros_like(ring_mask, dtype=bool)

                    # Angular Gap Analysis for Ld Arc
                    # -------------------------------
                    # Identify the dominant Ld arc by finding the largest angular gap
                    angles = np.arctan2(ring_yy - y, ring_xx - x)
                    angles = (angles + 2 * np.pi) % (2 * np.pi)
                    valid_angles = angles[valid_pixels]

                    if len(valid_angles) > 0:
                        sorted_angles = np.sort(valid_angles)
                        diffs = np.diff(sorted_angles)
                        wrap_diff = (sorted_angles[0] - sorted_angles[-1] + 2 * np.pi) % (2 * np.pi)
                        max_gap = np.max(diffs) if len(diffs) > 0 else 0
                        if wrap_diff > max_gap:
                            start_angle = sorted_angles[0]
                            end_angle = sorted_angles[-1] + 2 * np.pi
                        else:
                            max_gap_idx = np.argmax(diffs)
                            start_angle = sorted_angles[max_gap_idx + 1]
                            end_angle = sorted_angles[max_gap_idx] + 2 * np.pi
                        start_angle %= 2 * np.pi
                        end_angle %= 2 * np.pi
                        if start_angle < end_angle:
                            in_ld = (angles >= start_angle) & (angles <= end_angle)
                        else:
                            in_ld = (angles >= start_angle) | (angles <= end_angle)
                        Ld_arc_mask = np.zeros_like(image, dtype=bool)
                        Ld_arc_mask[ring_yy[in_ld], ring_xx[in_ld]] = True
                    else:
                        Ld_arc_mask = np.zeros_like(image, dtype=bool)

                    # Define Ld and Lo Masks
                    # ----------------------
                    # Assign Ld (mask2_i) and Lo (mask3_i) regions within the ring mask
                    mask2_i = final_ring_mask & Ld_arc_mask
                    mask3_i = final_ring_mask & ~Ld_arc_mask

                    # Postprocessing
                    # --------------
                    # Smooth and skeletonize masks to refine boundaries and sample intensities
                    mask2_i = closing(mask2_i, disk(3))  # Smooth Ld mask
                    mask3_i = closing(mask3_i, disk(3))  # Smooth Lo mask
                    mask2_i_skel = skeletonize(mask2_i) 
                    mask3_i_skel = skeletonize(mask3_i)
                    skeleton_width = 1  # Width for dilation to improve intensity sampling
                    mask2_i_skel_wide = binary_dilation(mask2_i_skel, disk(skeleton_width))
                    mask3_i_skel_wide = binary_dilation(mask3_i_skel, disk(skeleton_width))

                    # Accumulate masks for visualization
                    all_mask2 |= mask2_i_skel_wide
                    all_mask3 |= mask3_i_skel_wide

                    # Measurements
                    # ------------
                    # Calculate areas and intensities in Ld and Lo regions using the DNA channel
                    area_total = np.sum(final_ring_mask)
                    area_Ld = np.sum(mask2_i)
                    area_Lo = np.sum(mask3_i)
                    Ld_intensity = DNA_image[mask2_i_skel_wide]
                    Lo_intensity = DNA_image[mask3_i_skel_wide]
                    L_intensity = DNA_image[mask2_i_skel_wide + mask3_i_skel_wide]
                    mean_intensity_Ld = np.mean(Ld_intensity) if area_Ld > 0 else np.nan
                    max_intensity_Ld = np.max(Ld_intensity) if area_Ld > 0 else np.nan
                    mean_intensity_Lo = np.mean(Lo_intensity) if area_Lo > 0 else np.nan
                    max_intensity_Lo = np.max(Lo_intensity) if area_Lo > 0 else np.nan
                    mean_intensity_all = np.mean(L_intensity) if area_Lo > 0 else np.nan

                    # Store Results
                    # -------------
                    # Add measurements to a DataFrame for this vesicle
                    df_final = pd.concat([df_final, pd.DataFrame({
                        'vesicle_label': i,
                        'center_x': int(x),
                        'center_y': int(y),
                        'radius': int(r),
                        'enlarged_radius': enlarged_radius,
                        'inner_radius': inner_radius,
                        'area_total': area_total,
                        'area_Ld': area_Ld,
                        'area_Lo': area_Lo,
                        'library': library,
                        'mean_DNA(Lo)': mean_intensity_Lo,
                        'max_DNA(Lo)': max_intensity_Lo,
                        'mean_DNA(Ld)': mean_intensity_Ld,
                        'max_DNA(Ld)': max_intensity_Ld,    
                        'mean_DNA': mean_intensity_all
                    }, index=[0])], ignore_index=True)

                # Save Results to CSV
                # -------------------
                # Append results to a CSV file, with headers only for the first image
                if image_counter > 1:
                    df_final.to_csv(file_output, mode='a', index=True, header=False)
                else:
                    df_final.to_csv(file_output, mode='w', index=True, header=True)

                # Visualization
                # -------------
                # Generate plots showing detected vesicles, masks, and Ld/Lo regions
                fig, ax = plt.subplots(1, 4, figsize=(15, 5))
                ax[0].imshow(image, cmap='gray')
                for (x, y, r) in circles:
                    circ = plt.Circle((x, y), r * enlargement_factor, color='red', fill=False)
                    ax[0].add_patch(circ)
                ax[0].set_title('Detected GUVs')

                ax[1].imshow(labeled_mask, cmap='gray')
                ax[1].set_title('Final Ring Mask')

                ax[2].imshow(image, cmap='gray')
                ax[2].imshow(all_mask2, cmap='Greens', alpha=0.5)
                ax[2].imshow(all_mask3, cmap='Reds', alpha=0.5)
                ax[2].set_title('Ld(Green) & Lo(Red) in Ch.1(GUV)')

                # Label vesicles with final ring mask
                valid_labels = np.unique(labeled_mask)
                valid_labels = valid_labels[valid_labels > 0]
                for i in valid_labels:
                    x, y, r = circles[i-1]  # Index i-1 since circles start at 0, labels at 1
                    label_x = int(x + enlarged_radius + 10)
                    label_y = int(y)
                    ax[2].text(label_x, label_y, str(i), color='black', fontsize=8,
                               ha='left', va='center')

                ax[3].imshow(DNA_image, cmap='gray')
                ax[3].imshow(all_mask2, cmap='Greens', alpha=0.5)
                ax[3].imshow(all_mask3, cmap='Reds', alpha=0.5)
                ax[3].set_title('Ld(Green) & Lo(Red) in Ch.2(DNA)')
                for i in valid_labels:
                    x, y, r = circles[i-1]
                    label_x = int(x + enlarged_radius + 10)
                    label_y = int(y)
                    ax[3].text(label_x, label_y, str(i), color='black', fontsize=8,
                               ha='left', va='center')

                # Save the visualization as a TIFF file
                fig.savefig(os.path.join(newpath, library + image_filename[:-4] + "_analysis.tif"), dpi=300)
                plt.close()
print("All analyses are finished")